In [46]:
import pandas as pd
import yaml

In [47]:
repo_features = pd.read_csv('../data/new_repo_build_data/repo_features.csv')
repo_features.head()

,repo_name,build_tool,number_of_external_dependencies,number_of_java_files,number_of_lines_of_code,number_of_modules,number_of_unit_tests
0,Cetsoft-imcache.yaml,maven,39,120,10039,7,235
1,AmadeusITGroup-sonar-stash.yaml,maven,30,29,2817,1,174
2,BottegaIT-ddd-leaven-v2.yaml,maven,34,161,7744,1,4
3,Devskiller-friendly-id.yaml,maven,19,40,1075,5,33
4,BottegaIT-ddd-cqrs-sample.yaml,maven,38,147,7707,1,24


In [48]:
with open("../data/new_repo_build_data/jdk8_build_results_30k.yaml", "r") as file:
    build_results = yaml.safe_load(file)

In [59]:
with open("../data/migration_datasets/full_dataset.yaml", "r") as file:
    full_dataset = yaml.safe_load(file)

In [60]:
build_results[1]

{'build': {'build_success': True, 'errors': [], 'modules': {}, 'warnings': []},
 'repo_name': '0x727_DropLabTools',
 'test': {'build_success': True, 'errors': [], 'modules': {}, 'warnings': []}}

In [61]:
full_dataset[1]

{'commit': 'cc89b036d90403a9c6c98dd13800811a33a48962',
 'repo_name': 'magro/kryo-serializers'}

In [62]:
for item in full_dataset:
    if not item.get('repo_name'):
        print(item)
    repo_name = item['repo_name'].replace('/', '_')
    # search the test counts information from the jdk8 build results
    for build_result in build_results:
        if build_result['repo_name'].lower() == repo_name.lower():
            # obtain the test counts
            test_item= build_result.get('test')
            if test_item:
                test_results = test_item.get('tests')
                if test_results:
                    print(test_results)
                    item['test_count'] = test_results
    # search for repo features information from the repo_features.csv
    for index, row in repo_features.iterrows():
        
        if row['repo_name'].split('.')[0].lower() == item['repo_name'].replace('/', '-').lower():
            repo_item = { 'build_tool': row['build_tool'], 
                          'num_external_dependencies': row['number_of_external_dependencies'],
                         'num_java_files': row['number_of_java_files'],
                         'lines_of_code': row['number_of_lines_of_code'],
                         'num_modules': row['number_of_modules'],
                         'num_unit_tests': row['number_of_unit_tests'],
                         }
            item['repo_features'] = repo_item


{'tests_failed': 0, 'tests_run': 28, 'tests_skipped': 0, 'tests_successful': 28}
{'tests_failed': 0, 'tests_run': 381, 'tests_skipped': 0, 'tests_successful': 381}
{'tests_failed': 0, 'tests_run': 29, 'tests_skipped': 0, 'tests_successful': 29}


In [63]:
with open('../data/migration_datasets/full_dataset_modified.yaml', 'w') as file:
    yaml.dump(full_dataset, file, default_flow_style=False)

In [43]:
len(full_dataset)

284

In [44]:
repo_features.shape

(284, 7)